In [2]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization

from scipy import sparse
import pandas as pd
import os
from google.colab import drive

In [3]:
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
os.chdir("/content/drive/MyDrive/lendingClubLoanData/notebooks")

In [5]:
! ls

Exploration.ipynb  Preprocessing.ipynb	Train_Evaluate.ipynb


In [6]:
Ytrain = pd.read_csv("../../dataset/preprocessed_data/Ytrain.csv")
Ytest = pd.read_csv("../../dataset/preprocessed_data/Ytest.csv")
Xtrain_preprocessed = sparse.load_npz("../../dataset/preprocessed_data/Xtrain_preprocessed.npz")
Xtest_preprocessed = sparse.load_npz("../../dataset/preprocessed_data/Xtest_preprocessed.npz")

In [7]:
# Creation du modele

model = Sequential()
model.add(Input((Xtrain_preprocessed.shape[1],)))
model.add(Dense(units = 524 , activation = "relu"))
model.add(Dense(units = 256))
model.add(BatchNormalization())
model.add(Dense(units = 128, activation = "relu"))
model.add(Dense(units = 1, activation = "sigmoid"))


In [8]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 524)            │   576,490,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       134,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 576,658,577 (2.15 GB)

 Trainable params: 576,658,065 (2.15 GB)

 Non-trainable params: 512 (2.00 KB)

In [9]:
model.compile(
    loss = "binary_crossentropy",
    optimizer = "adam",
    metrics = ["accuracy"]
)

In [10]:
Ytrain.shape

(807186, 1)

In [11]:
history = model.fit(
    Xtrain_preprocessed, Ytrain,
    validation_data = (Xtest_preprocessed, Ytest),
    epochs = 2,
    verbose = True
)

Epoch 1/2


ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.12/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.12/dist-packages/ipykernel/kernelapp.py", line 712, in start

  File "/usr/local/lib/python3.12/dist-packages/tornado/platform/asyncio.py", line 211, in start

  File "/usr/lib/python3.12/asyncio/base_events.py", line 645, in run_forever

  File "/usr/lib/python3.12/asyncio/base_events.py", line 1999, in _run_once

  File "/usr/lib/python3.12/asyncio/events.py", line 88, in _run

  File "/usr/local/lib/python3.12/dist-packages/ipykernel/kernelbase.py", line 510, in dispatch_queue

  File "/usr/local/lib/python3.12/dist-pac